# Проверяем что сервис работает

In [10]:
server_url = "https://applied-python-hw-project.onrender.com"

In [11]:
import requests
data = requests.get(server_url + '/ping')
data.content

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

# Обучаем на дефолтных данных

In [4]:
data = requests.get(server_url + '/train')
data.content

b'{"message":"Model trained on default data and saved successfully"}'

# Обучаем на своих данных

In [5]:
from sklearn.datasets import load_iris
iris = load_iris()
X, y = iris.data, iris.target

data = requests.post(server_url + '/train_user',
        json= {
            'data': X.tolist(),
            'target': y.tolist()
})
data.content

b'{"message":"Model trained on user data  and saved successfully"}'

# Предикт 1 объекта и проверяем работу кэша

In [6]:
data = requests.post(server_url + '/predict',
        json= {
            'data': X.tolist()[0],
})
data.content # кэша нет

b'{"prediction":[0],"cashed":false}'

In [7]:
data = requests.post(server_url + '/predict',
        json= {
            'data': X.tolist()[0],
})
data.content  # кэш есть

b'{"prediction":[0],"cashed":true}'

# Предикт нескольких объектов и проверяем работу кэша

In [8]:
data = requests.post(server_url + '/predict/multiple',
        json= {
            'data': X.tolist()[:2],
})
data.content

b'{"predictions":[[0],[0]],"cashed":[true,false]}'

# Предикт 1 объекта в методе предназнченом для нескольких объектов

In [9]:
data = requests.post(server_url + '/predict/multiple',
        json= {
            'data': X.tolist()[0],
})
data.content # получаем Wrong data format

b'{"detail":"Wrong data format"}'